In [20]:
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
import seaborn as sns
from matplotlib import pyplot as plt
import sklearn.cross_validation
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
#because dataset is skewed we can try out logistic regression with balanced weights
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix
import tensorflow as tf

In [21]:
%pylab inline

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.cross_validation import train_test_split

try:
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.pipeline import make_pipeline
except ImportError:
    from sklearn_backports import PolynomialFeatures
    from sklearn_backports import make_pipeline

# ignore DeprecateWarnings by sklearn
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

Populating the interactive namespace from numpy and matplotlib


In [22]:
# Import SciKit Learn functions
from sklearn.metrics import roc_curve, auc , roc_auc_score, confusion_matrix, mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn import grid_search
from sklearn import tree

In [23]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(df_train.SeriousDlqin2yrs))
    plt.xticks(tick_marks, df_train.SeriousDlqin2yrs, rotation=45)
    plt.yticks(tick_marks, df_train.SeriousDlqin2yrs)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [24]:
df_train=pd.read_csv('train_ireg_amd_dmd.csv')
df_test=pd.read_csv('test_ireg_amd_dmd.csv')

In [25]:
df_train.head(10)

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0,1,0.766127,45,2,0.802982,9120.000000,13,0,6,0,2.0
1,1,0,0.957151,40,0,0.121876,2600.000000,4,0,0,0,1.0
2,2,0,0.658180,38,1,0.085113,3042.000000,2,1,0,0,0.0
3,3,0,0.233810,30,0,0.036050,3300.000000,5,0,0,0,0.0
4,4,0,0.907239,49,1,0.024926,63588.000000,7,0,1,0,0.0
5,5,0,0.213179,74,0,0.375607,3500.000000,3,0,1,0,1.0
6,6,0,0.305682,57,0,5710.000000,8738.957438,8,0,3,0,0.0
7,7,0,0.754464,39,0,0.209940,3500.000000,8,0,0,0,0.0
8,8,0,0.116951,27,0,46.000000,3045.627787,2,0,0,0,1.0
9,9,0,0.189169,57,0,0.606291,23684.000000,9,0,4,0,2.0


In [94]:
#Currently getting subset of data and just wrangling around and trying

Y_train=np.array(df_train.SeriousDlqin2yrs)
#features = ['RevolvingUtilizationOfUnsecuredLines', 'DebtRatio',
#            'MonthlyIncome', 'age', 'NumberOfTimes90DaysLate']
features=['RevolvingUtilizationOfUnsecuredLines','age','NumberOfTime30-59DaysPastDueNotWorse','DebtRatio','MonthlyIncome','NumberOfOpenCreditLinesAndLoans','NumberOfTimes90DaysLate','NumberRealEstateLoansOrLines','NumberOfTime60-89DaysPastDueNotWorse','NumberOfDependents']
X_train=np.array(df_train[features])

In [95]:
def dense_to_one_hot(labels_dense, num_classes=10):
  """Convert class labels from scalars to one-hot vectors."""
  num_labels = labels_dense.shape[0]
index_offset = numpy.arange(num_labels) * num_classes
  labels_one_hot = numpy.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot

In [96]:
nodes_hl1=1024
nodes_hl2=1024
nodes_hl3=1024
#nodes_hl3=20

nclasses=2
batch_size=4000


#placeholder is a special type of variable but its a dummy variable , its input node in the computational graph
#784 features
x=tf.placeholder('float',[None,10])
y=tf.placeholder('float')

hidden_1_layer={'weights':tf.Variable(tf.random_normal([10,nodes_hl1]))
                   ,'biases':tf.Variable(tf.random_normal([nodes_hl1]))
                   }
hidden_2_layer={'weights':tf.Variable(tf.random_normal([nodes_hl1,nodes_hl2]))
                   ,'biases':tf.Variable(tf.random_normal([nodes_hl2]))
                   }
    
hidden_3_layer={'weights':tf.Variable(tf.random_normal([nodes_hl2,nodes_hl3]))
               ,'biases':tf.Variable(tf.random_normal([nodes_hl3]))
               }
    
output_layer={'weights':tf.Variable(tf.random_normal([nodes_hl3,nclasses]))
                   ,'biases':tf.Variable(tf.random_normal([nclasses]))
                   }

def neural_network_model(data):
    #weights is tensorflow variable 
    
    #Wx+
    
    
    
    l1=tf.add(tf.matmul(data,hidden_1_layer['weights']),hidden_1_layer['biases'])
    l1=tf.nn.sigmoid(l1)#relu(l1)
    
    l2=tf.add(tf.matmul(l1,hidden_2_layer['weights']),hidden_2_layer['biases'])
    l2=tf.nn.sigmoid(l2)#relu(l2)


    l3=tf.add(tf.matmul(l2,hidden_3_layer['weights']),hidden_3_layer['biases'])
    l3=tf.nn.sigmoid(l3)
    
    output=tf.matmul(l3,output_layer['weights'])+output_layer['biases']

    return output



In [98]:
def train_neural_network(x):
    prediction=neural_network_model(x)
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(prediction,y) +
    0.01*tf.nn.l2_loss(hidden_1_layer['weights']) +
    0.01*tf.nn.l2_loss(hidden_1_layer['biases']) +
    0.01*tf.nn.l2_loss(hidden_2_layer['weights']) +
    0.01*tf.nn.l2_loss(hidden_2_layer['biases']) +
    0.01*tf.nn.l2_loss(hidden_3_layer['weights']) +
    0.01*tf.nn.l2_loss(hidden_3_layer['biases']) +
    0.01*tf.nn.l2_loss(output_layer['weights']) +
    0.01*tf.nn.l2_loss(output_layer['biases']))
    
    optimizer=tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)
    
    hm_epochs=10
    
    with tf.Session() as sess:
        # we have to initialize all the variables before run
        sess.run(tf.initialize_all_variables())
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            i=0
            while i < len(X_train):
                start = i
                end = i+batch_size
                batch_x = np.array(X_train[start:end])
                batch_y = np.array(Y_train[start:end])
                _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,y: batch_y})
                epoch_loss += c
                i+=batch_size
            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)
        correct=tf.equal(tf.arg_max(prediction,1),tf.arg_max(y,1)) #match indices
        accuracy=tf.reduce_mean(tf.cast(correct,'float'))
        #auc = tf.contrib.metrics.streaming_auc(a, b)
        prob=(prediction.eval(feed_dict={x:X_validation}))
        print prob[:,1].shape
        auc_score=roc_auc_score(Y_validation[:,1],prob[:,1])
        print(auc_score)
        print('Accuracy:',accuracy.eval({x:X_validation,y:Y_validation}))

In [99]:
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=0.15)

In [100]:
print Y_train.dtype

int64


In [101]:
Y_train=dense_to_one_hot(Y_train, num_classes=2)
Y_validation=dense_to_one_hot(Y_validation, num_classes=2)

In [102]:
print Y_train.shape

(127500, 2)


In [103]:
train_neural_network(x)

('Epoch', 1, 'completed out of', 10, 'loss:', 269166.79052734375)
('Epoch', 2, 'completed out of', 10, 'loss:', 163510.2509765625)
('Epoch', 3, 'completed out of', 10, 'loss:', 100311.3935546875)
('Epoch', 4, 'completed out of', 10, 'loss:', 62328.365966796875)
('Epoch', 5, 'completed out of', 10, 'loss:', 39186.697937011719)
('Epoch', 6, 'completed out of', 10, 'loss:', 24823.362976074219)
('Epoch', 7, 'completed out of', 10, 'loss:', 15867.108551025391)
('Epoch', 8, 'completed out of', 10, 'loss:', 10243.356018066406)
('Epoch', 9, 'completed out of', 10, 'loss:', 6685.5752716064453)
('Epoch', 10, 'completed out of', 10, 'loss:', 4434.7273254394531)
(22500,)
0.409022627873
('Accuracy:', 0.93502223)


In [65]:
prediction_test=neural_network_model(x)

Tensor("Placeholder_8:0", shape=(?, 10), dtype=float32)


In [64]:
neural_network_model(x)

Tensor("Placeholder_8:0", shape=(?, 10), dtype=float32)


<tf.Tensor 'add_21:0' shape=(?, 2) dtype=float32>

In [18]:
df_test.head()

,Unnamed: 0,index,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0,1,NaN,0.885519,43,0,0.177513,5700.0,4,0,0,0,0.0
1,1,2,NaN,0.463295,57,0,0.527237,9141.0,15,0,4,0,2.0
2,2,3,NaN,0.043275,59,0,0.687648,5083.0,12,0,1,0,2.0
3,3,4,NaN,0.280308,38,1,0.925961,3200.0,7,0,2,0,0.0
4,4,5,NaN,1.000000,27,0,0.019917,3865.0,4,0,0,0,1.0
